In [1]:
import os
import torch

from config import Config
from models.baseline import BSL
from dataset import get_loader
from evaluation.valid import validate


os.environ["CUDA_VISIBLE_DEVICES"] = "0"
val_sets = 'CoCA+CoSOD3k+CoSal2015'
testsets = val_sets

config = Config()
# Prepare dataset
test_loaders = {}
for testset in testsets.split('+'):
    test_loader = get_loader(
        os.path.join(config.root_dir, 'images', testset), os.path.join(config.root_dir, 'gts', testset),
        config.size, 1, istrain=False, shuffle=False, num_workers=int(config.num_workers//2), pin=True
    )
    test_loaders[testset] = test_loader
# Init model
model = BSL().to(config.device)

In [2]:
from glob import glob
import shutil


ckpts = glob('ckpt/tmp-INS/*.pth')
valid_only_S = True

for ckpt in ckpts:
    val_save = '__'.join(ckpt.split('/')[-2:]).rstrip('.pth')
    if os.path.exists(val_save):
        shutil.rmtree(val_save)
    print('Loading', ckpt)
    model.load_state_dict(torch.load(ckpt))
    measures = validate(model, test_loaders, val_save, val_sets, valid_only_S=valid_only_S)
    for idx_dataset, (testset, scores_per_dataset) in enumerate(zip(testsets.split('+'), measures)):
        if valid_only_S:
            sm = scores_per_dataset[0]
            print('Dataset:', testset)
            print('\tSm: {:.4f}.'.format(
                sm.round(3)
            ))
        else:
            sm, mae, fm, wfm, em = scores_per_dataset
            print('Dataset:', testset)
            print('\tEmax: {:.4f}, \tSm: {:.4f}, \tFmax: {:.4f}, \tMAE: {:.4f}.'.format(
                em['curve'].max().round(3), sm.round(3), fm['curve'].max().round(3), mae.round(3)
            ))

Loading ckpt/tmp-INS/ep100.pth
Validating ..., CoCA, CoSal2015, CoSOD3k
Dataset: CoCA
	Sm: 0.7200.
Dataset: CoSal2015
	Sm: 0.9110.
Dataset: CoSOD3k
	Sm: 0.8920.
Loading ckpt/tmp-INS/ep110.pth
Validating ..., CoCA, CoSal2015, CoSOD3k
Dataset: CoCA
	Sm: 0.7140.
Dataset: CoSal2015
	Sm: 0.9060.
Dataset: CoSOD3k
	Sm: 0.8910.
Loading ckpt/tmp-INS/ep120.pth
Validating ..., CoCA